In [2]:
import pandas as pd
from scipy.stats import entropy


4

In [75]:
df = pd.read_csv("/scratch/venia/web2wiki/data/web_content/iterative_coding_sample/cleaner_data.csv")

In [67]:
df = df.drop_duplicates(subset = ["url","wiki_links"])

In [69]:

def addlabels(x,y):
    for i in range(len(x)):
        plt.text(i,y[i] + 0.01, int(round(100*y[i],0)), ha = "center", fontsize = 12)

In [78]:
import tldextract
import urllib

def extract_subdomain(x):
    if len(tldextract.extract(x).subdomain) > 1:
        y = tldextract.extract(x).subdomain +"."+tldextract.extract(x).registered_domain
    else:
        y = tldextract.extract(x).registered_domain
    return y

def normalise_title(title):
    """ Replace _ with space, remove anchor, capitalize """
    title = title.split("/")[-1]
    title = title.split("#")[0]
    title = urllib.parse.unquote(title)
    title = title.strip()
    if len(title) > 0:
        title = title[0].upper() + title[1:]
    n_title = title.replace("_", " ")
    # if '#' in n_title:
    #     n_title = n_title.split('#')[0]
    return n_title

In [79]:
df["link_type"] = df["wiki_links"].apply(lambda x: 1 if "wikipedia" in str(x) else 0)
df = df.dropna(subset="wiki_links")
df = df[df["wiki_links"].apply(lambda x: True if "http" in str(x) else False)]

In [103]:
dff = df[df["link_type"] == 1]
dff["title"] = dff["wiki_links"].map(normalise_title)

/tmp/ipykernel_2938310/1884108815.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dff["title"] = dff["wiki_links"].map(normalise_title)


# calculate entropy

In [82]:
# dff = df[df["domain_count"]<1000]

In [105]:
# dff = dff.drop_duplicates(subset = ["title","domain"])
t = dff.groupby(["order","title"])["url"].count().sort_values().reset_index()
t["total_count"] = t.groupby("order")["url"].transform("sum")
t["p"] = t["url"] / t["total_count"]
zero = t[t["order"] == 0]
one = t[t["order"] == 1]
two = t[t["order"] == 2]
print(entropy(zero.sample(len(two))["p"]))
print(entropy(one.sample(len(two))["p"]))
print(entropy(two["p"]))

7.652458379810653
8.472254284474493
8.414428831157824
